# 1.3 voor een orgaan valt het aantal mandatarissen op elk moment tussen de min en max aantallen

We load Javascript libraries to i) retrieve data (bindings) from a bestuurseenheid, ii) validate the document, and iii) display the JSON result.

In [1]:
import { getBestuurseenheidUrl } from "./utils.ts";
import {  getBindingsFromTurtleContent, validateDocument, fetchDocument } from 'npm:@lblod/lib-decision-validation';
import { display } from "https://deno.land/x/display/mod.ts";

const bestuurseenheidUuid = 'd168033a9bac278fa744c425e078eeabd304397f953feaaf5327b4e039aecacb';
const bestuurseenheidUrl = getBestuurseenheidUrl(bestuurseenheidUuid);
const bestuurseenheidBindings = await fetchDocument(bestuurseenheidUrl);

Below, we can find the SPARQL template query and SHACL shape for the 1.3 use case.
The Turtle string is converted into bindings.

In [2]:
const mandataris_1_3_query: string = `
    PREFIX besluit: <http://data.vlaanderen.be/ns/besluit#>
    PREFIX mandaat: <http://data.vlaanderen.be/ns/mandaat#>
    PREFIX org: <http://www.w3.org/ns/org#>
    PREFIX lmb: <http://lblod.data.gift/vocabularies/lmb/>

    select DISTINCT ($this as ?this) ?value
    where {
        $this a besluit:Bestuursorgaan ;
                    org:hasPost ?mandaat .
                    
            ?mandaat lmb:minAantalHouders ?min ;
                        lmb:maxAantalHouders ?max .
        {
            select ?startPunt ?mandaat (count(distinct ?mandataris) as ?aantalMandatarissen)
            where {
                ?mandataris a mandaat:Mandataris ;
                    org:holds ?mandaat ;
                    mandaat:status ?mandatarisStatus .
                    mandaat:start ?startMandataris .
                OPTIONAL {
                    ?mandataris mandaat:einde ?eindeMandataris .
                }

                VALUES ?mandatarisStatus { <http://data.vlaanderen.be/id/concept/MandatarisStatusCode/e1ca6edd-55e1-4288-92a5-53f4cf71946a> 
                    <http://data.vlaanderen.be/id/concept/MandatarisStatusCode/21063a5b-912c-4241-841c-cc7fb3c73e75>
                }
                        
                {
                    select distinct ?startPunt ?mandaat
                    where {
                        $this a besluit:Bestuursorgaan ;
                            org:hasPost ?mandaat ;
                            mandaat:bindingStart ?startBestuursorgaan .

                        OPTIONAL {
                            $this mandaat:bindingEinde ?eindeBestuursorgaan .
                        }

                        ?mandataris org:holds ?mandaat ;
                                    mandaat:start ?startPunt .

                        BIND(strdt(substr(str(?startPunt), 1, 10), xsd:date) as ?startPuntDate)
                        BIND(strdt(substr(str(?startBestuursorgaan), 1, 10), xsd:date) as ?startBestuursorgaanDate)
                        BIND(strdt(substr(str(?eindeBestuursorgaan), 1, 10), xsd:date) as ?eindeBestuursorgaanDate)
                        
                        FILTER (?startPuntDate >= ?startBestuursorgaanDate)
                        FILTER (if(bound(?eindeBestuursorgaan), ?startPuntDate <= ?eindeBestuursorgaanDate, true))
                    }
                }

                FILTER (?startMandataris <= ?startPunt && ?eindeMandataris >= ?startPunt)
            }
            GROUP BY ?startPunt ?mandaat
        }
        FILTER (?aantalMandatarissen < ?min || ?aantalMandatarissen > ?max)

        BIND ("Dit bestuursorgaan bevat niet op elk moment het aantal effectieve/waarnemende mandatarissen tussen de min en max aantallen." as ?value)
    }`;
    
const mandataris_1_3_shape: string = `
@prefix sh: <http://www.w3.org/ns/shacl#> .

<http://example.org/mandataris_1_3_blueprint>
  a sh:NodeShape ;
  sh:targetClass <http://data.vlaanderen.be/ns/besluit#Bestuursorgaan> ;
  sh:sparql [
		sh:select """${mandataris_1_3_query}""" ;
        sh:message 'Dit bestuursorgaan bevat niet op elk moment het aantal effectieve/waarnemende mandatarissen tussen de min en max aantallen.'
] .`;

const mandataris_1_3_bindings: Bindings[] = await getBindingsFromTurtleContent(mandataris_1_3_shape);

To test the shape, we retrieved a subject from the triple store that is applicable. In this case, we choose a bestuursorgaan URI.
Then, the template query from above is applied on this subject.
Next, a Comunica URL is generated to debug the results of this validation.

In [3]:
// Test subject
const subject = 'http://data.lblod.info/id/bestuursorganen/9b0bc40a5992682b3482a74badb0ac4bc2f270b741dbafe20d39717f7c666863';
const mandataris_1_3_subject_query = mandataris_1_3_query.replaceAll('$this', `<${subject}>`);

const mandataris_1_3_comunica = `https://query.linkeddatafragments.org/#datasources=${encodeURIComponent(bestuurseenheidUrl)}&query=${encodeURIComponent(mandataris_1_3_subject_query)}`;
console.log(mandataris_1_3_comunica);

https://query.linkeddatafragments.org/#datasources=http%3A%2F%2Flocalhost%3A8890%2Fsparql%2F%3Fdefault-graph-uri%3D%26query%3D%250A%2520%2520%2520%2520construct%2520%257B%250A%2520%2520%2520%2520%2520%2520%2520%2520%253Fs%2520%253Fp%2520%253Fo%2520.%250A%2520%2520%2520%2520%257D%250A%2520%2520%2520%2520where%2520%257B%250A%2520%2520%2520%2520%2520%2520%2520%2520graph%2520%253Chttp%253A%252F%252Fmu.semte.ch%252Fgraphs%252Forganizations%252Fd168033a9bac278fa744c425e078eeabd304397f953feaaf5327b4e039aecacb%252FLoketLB-mandaatGebruiker%253E%2520%257B%250A%2520%2520%2520%2520%2520%2520%2520%2520%2520%2520%2520%2520%253Fs%2520%253Fp%2520%253Fo%2520.%250A%2520%2520%2520%2520%2520%2520%2520%2520%257D%250A%2520%2520%2520%2520%257D%250A%2520%2520%2520%2520&query=%0A%20%20%20%20PREFIX%20besluit%3A%20%3Chttp%3A%2F%2Fdata.vlaanderen.be%2Fns%2Fbesluit%23%3E%0A%20%20%20%20PREFIX%20mandaat%3A%20%3Chttp%3A%2F%2Fdata.vlaanderen.be%2Fns%2Fmandaat%23%3E%0A%20%20%20%20PREFIX%20org%3A%20%3Chttp%3A%2F%2Fwww.w

The full validation on all mandatarissen happens below:

In [4]:
const mandataris_1_3_validationReport = await validateDocument(bestuurseenheidBindings, mandataris_1_3_bindings);
await display({ "application/json": mandataris_1_3_validationReport},{ raw: true });